Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# Provided Code

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

# Train/Val/Test Split

In [0]:
#imports
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.linear_model import LogisticRegression

In [0]:
#making a new year column in order to split the dataset
df['Year'] = df['Date'].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True)).apply(lambda x: x.year)

In [0]:
#splitting the dataset
train = df[df['Year'] <= 2016]
val = df[df['Year'] == 2017]
test = df[df['Year'] >= 2018]

# Baseline Accuracy

In [10]:
#finding the baseline accuracy
target = 'Great'
y_train = train[target]
y_val = val[target]

majority_class = y_train.mode()[0]

#train
y_pred_train = [majority_class] * len(y_train)
baseline_accuracy_train = accuracy_score(y_train, y_pred_train)

#val
y_pred_val = [majority_class] * len(y_val)
baseline_accuracy_val = accuracy_score(y_val, y_pred_val)

print(f'Majority Class:   {majority_class} \n')
print(f'Train Baseline Accuracy Score:   {baseline_accuracy_train}')
print(f'Val Baseline Accuracy Score:     {baseline_accuracy_val}')

Majority Class:   False 

Train Baseline Accuracy Score:   0.5906040268456376
Val Baseline Accuracy Score:     0.5529411764705883


# Cleaning The Data

In [0]:
#dropping columns that are either empty, the target or that i just dont want
empty_columns = ['Mass (g)', 'Density (g/mL)']
date_data = ['Date', 'Year']
features = train.columns.drop([target] + empty_columns + date_data)

In [12]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great,Year
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2016
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2016
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2016
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2016
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,2016


In [0]:
#a lot of the features appear to be a binary value of if an ingredient was present or not
#this is coded as x, X or nan
#im assuming that x and X are the same

binary_columns = [
 'Unreliable',
 'NonSD',
 'Beef',
 'Pico',
 'Guac',
 'Cheese',
 'Fries',
 'Sour cream',
 'Pork',
 'Chicken',
 'Shrimp',
 'Fish',
 'Rice',
 'Beans',
 'Lettuce',
 'Tomato',
 'Bell peper',
 'Carrots',
 'Cabbage',
 'Sauce',
 'Salsa.1',
 'Cilantro',
 'Onion',
 'Taquito',
 'Pineapple',
 'Ham',
 'Chile relleno',
 'Nopales',
 'Lobster',
 'Queso',
 'Egg',
 'Mushroom',
 'Bacon',
 'Sushi',
 'Avocado',
 'Corn',
 'Zucchini']

#train
X_train = train[features]
X_train_binary = X_train.copy()
for column in binary_columns:
    X_train_binary[column] = X_train_binary[column].apply(
            lambda x: str(x).lower()).apply(lambda x: 1 if x == 'x' else 0)

#val
X_val = val[features]
X_val_binary = X_val.copy()
for column in binary_columns:
    X_val_binary[column] = X_val_binary[column].apply(
            lambda x: str(x).lower()).apply(lambda x: 1 if x == 'x' else 0)

#test
X_test = test[features]
X_test_binary = X_test.copy()
for column in binary_columns:
    X_test_binary[column] = X_test_binary[column].apply(
            lambda x: str(x).lower()).apply(lambda x: 1 if x == 'x' else 0)


In [0]:
#i think it is important to keep in the data if a yelp or google review was made or not
#going to turn the reviews into strings and one hot encode

encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)

#train
X_train_pre_one_hot = X_train_binary
X_train_pre_one_hot['Yelp'] = X_train_pre_one_hot['Yelp'].apply(lambda x: str(x))
X_train_pre_one_hot['Google'] = X_train_pre_one_hot['Google'].apply(lambda x: str(x))

X_train_post_one_hot = encoder.fit_transform(X_train_pre_one_hot)

#val
X_val_pre_one_hot = X_val_binary
X_val_pre_one_hot['Yelp'] = X_val_pre_one_hot['Yelp'].apply(lambda x: str(x))
X_val_pre_one_hot['Google'] = X_val_pre_one_hot['Google'].apply(lambda x: str(x))

X_val_post_one_hot = encoder.transform(X_val_pre_one_hot)

#test
X_test_pre_one_hot = X_test_binary
X_test_pre_one_hot['Yelp'] = X_test_pre_one_hot['Yelp'].apply(lambda x: str(x))
X_test_pre_one_hot['Google'] = X_test_pre_one_hot['Google'].apply(lambda x: str(x))

X_test_post_one_hot = encoder.transform(X_test_pre_one_hot)

In [0]:
#filling in missing data with the mean of that column

imputer = SimpleImputer()

#train
X_train_post_impute = imputer.fit_transform(X_train_post_one_hot)
X_train_post_impute = pd.DataFrame(X_train_post_impute, columns=X_train_post_one_hot.columns)


#val
X_val_post_impute = imputer.transform(X_val_post_one_hot)
X_val_post_impute = pd.DataFrame(X_val_post_impute, columns=X_val_post_one_hot.columns)

#test
X_test_post_impute = imputer.transform(X_test_post_one_hot)
X_test_post_impute = pd.DataFrame(X_test_post_impute, columns=X_test_post_one_hot.columns)

In [20]:
y_train

0      False
1      False
2      False
3      False
4       True
       ...  
296    False
297    False
298    False
299    False
300    False
Name: Great, Length: 298, dtype: bool

In [21]:
log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train_post_impute, y_train)
print('Validation Accuracy', log_reg.score(X_train_post_impute, y_train))

Validation Accuracy 0.9194630872483222


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [22]:
y_val = val[target]
print('Validation Accuracy', log_reg.score(X_val_post_impute, y_val))

Validation Accuracy 0.8117647058823529


In [23]:
y_test = test[target]
print('Validation Accuracy', log_reg.score(X_test_post_impute, y_test))

Validation Accuracy 0.7631578947368421
